# Decoding Sector Valuation Dynamics

Exploring the Nexus of Company Fundamentals, Economic Indicators, and Equity Valuations via Predictive Analytics


## Data Preprocessing


In [1]:
import pandas as pd
import numpy as np

# Constants (525 weeks or about 10 years)
DATE_START = pd.to_datetime('2014-01-25')
DATE_END = pd.to_datetime('2024-02-17')
# Offset the start date by 2 quarters
# Useful for calculating the % change before the start date
DATE_START_OFFSET = DATE_START - pd.DateOffset(weeks=26)

### Retrieve Data

#### Financials


In [2]:
# Load financials and assign cap

def load_financials(cap: str) -> pd.DataFrame:
    df: pd.DataFrame = pd.read_csv(f'data/{cap}/financials_q.csv')
    df['cap'] = cap
    return df


df: pd.DataFrame = pd.concat([load_financials(cap)
                             for cap in ['large', 'mid', 'small']])
df['asOfDate'] = pd.to_datetime(df['asOfDate'])

# Insert GICS sector for each symbol
df_gics: pd.DataFrame = pd.concat([pd.read_csv(
    f'data/{cap}/tickers+GICS.csv') for cap in ['large', 'mid', 'small']])
df = pd.merge(df, df_gics, left_on='symbol', right_on='Symbol')
%xdel df_gics

df.drop(columns=['Symbol'], inplace=True)

# Show empty GICS Sub-Industry Companies
display(df[df['GICS Sub-Industry'].isnull()][['symbol', 'GICS Sector']].drop_duplicates())

print('Total Filings:', df.shape)
df.tail()

,symbol,GICS Sector
61741,ALLY,Financials
73999,FNF,Financials
74631,GDDY,Information Technology
75092,GLPI,Real Estate
82755,MORN,Financials
83377,MTN,Consumer Discretionary
89133,PR,Energy
89902,RBC,Industrials
94249,ST,Industrials
99277,WFRD,Energy


Total Filings: (159841, 345)


,symbol,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccruedInterestReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AdjustedGeographySegmentData,...,TreasuryStock,UnrealizedGainLoss,UnrealizedGainLossOnInvestmentSecurities,WorkInProcess,WorkingCapital,WriteOff,cap,Security,GICS Sector,GICS Sub-Industry
159836,ZEUS,2022-09-30,3M,USD,167081000.0,279344000.0,NaN,-293845000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,574409000.0,NaN,small,NaN,Materials,Steel
159837,ZEUS,2022-12-31,3M,USD,101446000.0,219789000.0,NaN,-299241000.0,NaN,NaN,...,0.0,NaN,NaN,NaN,493404000.0,NaN,small,NaN,Materials,Steel
159838,ZEUS,2023-03-31,3M,USD,142608000.0,236844000.0,NaN,-301696000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,481492000.0,NaN,small,NaN,Materials,Steel
159839,ZEUS,2023-06-30,3M,USD,124087000.0,227992000.0,NaN,-307118000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,481824000.0,NaN,small,NaN,Materials,Steel
159840,ZEUS,2023-09-30,3M,USD,127671000.0,227847000.0,NaN,-312384000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,455528000.0,NaN,small,NaN,Materials,Steel


In [3]:
# Several are missing GICS Sub-Industry
# We will fill these in manually using revenue information from their 10-K filings

# According to ALLY's 10-K, they derive the majority of their revenue from Automotive Finance and Insurance operations (Consumer Finance)
df.loc[df['symbol'] == 'ALLY', 'GICS Sub-Industry'] = 'Consumer Finance'

# FNF derives the majority of its revenue from Title Insurance according to their 10-K
df.loc[df['symbol'] == 'FNF', 'GICS Sub-Industry'] = 'Property & Casualty Insurance'

# GDDY derives the majority of its revenue from Applications and Commerce (proprietary website software and POS fees)
df.loc[df['symbol'] == 'GDDY', 'GICS Sub-Industry'] = 'Internet Services & Infrastructure'

# GLPI is a REIT specializing in Gaming and Leisure (Casinos, and Resorts/Spas)
df.loc[df['symbol'] == 'GLPI', 'GICS Sub-Industry'] = 'Hotel & Resort REITs'

# MORN derives the majority of its revenue from Investment Research and Management data
df.loc[df['symbol'] == 'MORN', 'GICS Sub-Industry'] = 'Financial Exchanges & Data'

# MTN derives the majority of its revenue from Ski tickets at their resorts
df.loc[df['symbol'] == 'MTN', 'GICS Sub-Industry'] = 'Leisure Facilities'

# PR derives the majority of its revenue from Oil & Gas sales
df.loc[df['symbol'] == 'PR', 'GICS Sub-Industry'] = 'Oil & Gas Exploration & Productions'

# RBC derives the majority of its revenue from bearings and precision components
df.loc[df['symbol'] == 'RBC',
       'GICS Sub-Industry'] = 'Industrial Machinery & Supplies & Components'

# ST has a majority of its revenue from sensor based products
df.loc[df['symbol'] == 'ST', 'GICS Sub-Industry'] = 'Electronic Components'

# WFRD derives the majority of its revenue from their oil & gas related services and equipment
df.loc[df['symbol'] == 'WFRD', 'GICS Sub-Industry'] = 'Oil & Gas Equipment & Services' 

# GICS Information
df_gics_info: pd.DataFrame = pd.read_csv('data/gics.csv')

# Check for differences between GICS Sub-Industry names
display(df[~df['GICS Sub-Industry'].isin(df_gics_info['GICS Sub-Industry'])][['GICS Sector', 'GICS Sub-Industry']].drop_duplicates())

,GICS Sector,GICS Sub-Industry
63216,Materials,Metal & Glass Containers
64340,Materials,Paper Packaging
64424,Consumer Staples,Hypermarkets & Super Centers
66453,Consumer Staples,Soft Drinks
68770,Consumer Staples,Personal Products
69952,Consumer Staples,Agricultural Products
70098,Industrials,Industrial Machinery
70515,Consumer Discretionary,Specialty Stores
74326,Industrials,Construction Machinery & Heavy Trucks
79108,Industrials,Marine


Several Sub-Industries appear to have slightly different names between the two datasets.

In [4]:
# Metal & Glass Containers to Metal, Glass & Plastic Containers
df.loc[df['GICS Sub-Industry'] == 'Metal & Glass Containers', 'GICS Sub-Industry'] = 'Metal, Glass & Plastic Containers'
# Paper Packaging to Paper & Plastic Packaging Products & Materials
df.loc[df['GICS Sub-Industry'] == 'Paper Packaging', 'GICS Sub-Industry'] = 'Paper & Plastic Packaging Products & Materials'

# Check Hypermarkets & Super Centers tickers
df[df['GICS Sub-Industry'] == 'Hypermarkets & Super Centers']['symbol'].unique()

array(['BJ', 'PSMT'], dtype=object)

In [5]:
# Hypermarkets & Super Centers to Consumer Staples Merchandise Retail
df.loc[df['GICS Sub-Industry'] == 'Hypermarkets & Super Centers', 'GICS Sub-Industry'] = 'Consumer Staples Merchandise Retail'

# Soft Drinks to Soft Drinks & Non-alcoholic Beverages
df.loc[df['GICS Sub-Industry'] == 'Soft Drinks', 'GICS Sub-Industry'] = 'Soft Drinks & Non-alcoholic Beverages'

# Personal Products to Personal Care Products
df.loc[df['GICS Sub-Industry'] == 'Personal Products', 'GICS Sub-Industry'] = 'Personal Care Products'

# Agricultural Products to Agricultural Products & Services
df.loc[df['GICS Sub-Industry'] == 'Agricultural Products', 'GICS Sub-Industry'] = 'Agricultural Products & Services'

# Industrial Machinery to Industrial Machinery & Supplies & Components
df.loc[df['GICS Sub-Industry'] == 'Industrial Machinery', 'GICS Sub-Industry'] = 'Industrial Machinery & Supplies & Components'

# Specialty Stores to Other Specialty Retail
df.loc[df['GICS Sub-Industry'] == 'Specialty Stores', 'GICS Sub-Industry'] = 'Other Specialty Retail'

# Construction Machinery & Heavy Trucks to Construction Machinery & Heavy Transportation Equipment
df.loc[df['GICS Sub-Industry'] == 'Construction Machinery & Heavy Trucks', 'GICS Sub-Industry'] = 'Construction Machinery & Heavy Transportation Equipment'

# Marine to Marine Transportation
df.loc[df['GICS Sub-Industry'] == 'Marine', 'GICS Sub-Industry'] = 'Marine Transportation'

# Semiconductor Equipment to Semiconductor Materials & Equipment
df.loc[df['GICS Sub-Industry'] == 'Semiconductor Equipment', 'GICS Sub-Industry'] = 'Semiconductor Materials & Equipment'

# Oil & Gas Exploration & Productions to Oil & Gas Exploration & Production
df.loc[df['GICS Sub-Industry'] == 'Oil & Gas Exploration & Productions', 'GICS Sub-Industry'] = 'Oil & Gas Exploration & Production'

# Airlines to Passenger Airlines
df.loc[df['GICS Sub-Industry'] == 'Airlines', 'GICS Sub-Industry'] = 'Passenger Airlines'

# Auto Parts & Equipment to Automotive Parts & Equipment
df.loc[df['GICS Sub-Industry'] == 'Auto Parts & Equipment', 'GICS Sub-Industry'] = 'Automotive Parts & Equipment'

# Specialized REITs to Other Specialized REITs
df.loc[df['GICS Sub-Industry'] == 'Specialized REITs', 'GICS Sub-Industry'] = 'Other Specialized REITs'

# Merge
df = pd.merge(df, df_gics_info, on='GICS Sub-Industry', how='left')

# Drop extra GICS Sector
df.drop(columns=['GICS Sector_y'], inplace=True)
df.rename(columns={'GICS Sector_x': 'GICS Sector'}, inplace=True)

# Display number of nulls
with pd.option_context('display.max_rows', None):
    display(df.isnull().sum())

symbol                                                               0
asOfDate                                                             0
periodType                                                           0
currencyCode                                                         0
AccountsPayable                                                  31151
AccountsReceivable                                               48688
AccruedInterestReceivable                                       158538
AccumulatedDepreciation                                          55623
AdditionalPaidInCapital                                          47167
AdjustedGeographySegmentData                                    159795
AllowanceForDoubtfulAccountsReceivable                          126321
Amortization                                                    137484
AmortizationCashFlow                                            130281
AmortizationOfIntangibles                                       130337
Amorti

In [6]:
# View sectors, sector groups, and Industry, and sub-industry
with pd.option_context('display.max_rows', None):
	display(df.groupby(['GICS Sector', 'GICS Industry Group', 'GICS Industry', 'GICS Sub-Industry']).size())

GICS Sector             GICS Industry Group                             GICS Industry                                          GICS Sub-Industry                                      
Communication Services  Media & Entertainment                           Entertainment                                          Interactive Home Entertainment                               247
                                                                                                                               Movies & Entertainment                                       650
                                                                        Interactive Media & Services                           Interactive Media & Services                                 752
                                                                        Media                                                  Advertising                                                  406
                                                 

Several Industry groups have been assigned to the wrong sector for some reason.

In [7]:
# Assign any improper GICS Sectors
sector_map = df_gics_info[['GICS Industry Group', 'GICS Sector']].drop_duplicates().set_index('GICS Industry Group').to_dict()['GICS Sector']
df['GICS Sector'] = df['GICS Industry Group'].map(sector_map)
with pd.option_context('display.max_rows', None):
	display(df.groupby(['GICS Sector', 'GICS Industry Group', 'GICS Industry', 'GICS Sub-Industry']).size())

GICS Sector             GICS Industry Group                             GICS Industry                                          GICS Sub-Industry                                      
Communication Services  Media & Entertainment                           Entertainment                                          Interactive Home Entertainment                               247
                                                                                                                               Movies & Entertainment                                       805
                                                                        Interactive Media & Services                           Interactive Media & Services                                 752
                                                                        Media                                                  Advertising                                                  406
                                                 

#### Earnings


In [8]:
# Load Earnings
df_earn = pd.concat(
    [pd.read_csv(f'data/{cap}/earnings.csv') for cap in ['large', 'mid', 'small']])

df_earn['Earnings Date'] = df_earn['date'] + ' ' + df_earn['time']
df_earn.drop(columns=['date', 'time'], inplace=True)

df_earn['Earnings Date'] = pd.to_datetime(df_earn['Earnings Date'])
df_earn['asOfDate'] = pd.to_datetime(df_earn['Earnings Date'].dt.date)

# Merge earnings with financials
# SEC requires 40-45 days after the end of the quarter to file 10-Q 
# and earnings calls are typically before or same day as 10-Q filings,
# we'll use 50 days
df = pd.merge_asof(df.sort_values('asOfDate'), df_earn.sort_values(
    'asOfDate'), on='asOfDate', by='symbol', tolerance=pd.Timedelta(days=50), direction='forward')
%xdel df_earn

print('Total Filings:', df.shape)
display(df.tail())
# Get the number of companies in each cap
print('\nCompany Counts:\n', df.groupby('cap')['symbol'].nunique())
# Get the number filings in each cap
print('\nFiling Counts:\n', df.groupby('cap').size())

Total Filings: (159841, 353)


,symbol,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccruedInterestReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AdjustedGeographySegmentData,...,GICS Sector,GICS Sub-Industry,GICS Sector ID,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,title,Earnings Date
159836,MDLZ,2023-12-31,3M,USD,8.321000e+09,3.634000e+09,NaN,-7.996000e+09,3.221600e+10,NaN,...,Consumer Staples,Packaged Foods & Meats,30,3020,"Food, Beverage & Tobacco",302020.0,Food Products,30202030.0,Mondelez International,2024-01-30 16:05:00
159837,CRNC,2023-12-31,3M,USD,1.917900e+07,5.869300e+07,NaN,NaN,1.064022e+09,NaN,...,Information Technology,Application Software,45,4510,Software & Services,451030.0,Software,45103010.0,Cerence,2024-02-06 07:00:00
159838,META,2023-12-31,3M,USD,4.849000e+09,1.616900e+10,NaN,-3.313400e+10,7.325300e+10,NaN,...,Communication Services,Interactive Media & Services,50,5020,Media & Entertainment,502030.0,Interactive Media & Services,50203010.0,Meta Platforms,2024-02-01 16:05:00
159839,LOPE,2023-12-31,3M,USD,1.767600e+07,7.881100e+07,NaN,-1.704150e+08,3.225120e+08,NaN,...,Consumer Discretionary,Education Services,25,2530,Consumer Services,253020.0,Diversified Consumer Services,25302010.0,Grand Canyon Education,2024-02-13 16:05:00
159840,CRS,2023-12-31,3M,USD,3.134000e+08,5.084000e+08,NaN,NaN,3.279000e+08,NaN,...,Materials,Steel,15,1510,Materials,151040.0,Metals & Mining,15104050.0,Carpenter Tech,2024-01-25 08:00:00



Company Counts:
 cap
large    503
mid      401
small    602
Name: symbol, dtype: int64

Filing Counts:
 cap
large    60852
mid      40274
small    58715
dtype: int64


### Categorical Columns


In [9]:
cat_cols = ['symbol', 'cap', 'GICS Sector ID', 'GICS Sector', 'GICS Industry Group ID', 'GICS Industry Group', 'GICS Industry ID', 'GICS Industry', 'GICS Sub-Industry ID', 'GICS Sub-Industry']
df[cat_cols] = df[cat_cols].apply(lambda x: x.astype('category'))

### Clean


In [10]:
import json

# Drop earnings date outside of the date range
df = df[(df['Earnings Date'] >= DATE_START_OFFSET) &
        (df['Earnings Date'] <= DATE_END)]
print('Filings in Range:', df.shape)

# Remove filings where the company name is not in the company names list,
# tickers may change hands between companies. Known names for each company
# was collected from SEC filings, we'll also compare with the title of the
# company from the earnings data in case the company's common name is not
# the same as the official name registered with the SEC
companies = {}
for cap in ['large', 'mid', 'small']:
    with open(f'data/{cap}/company_names.json') as f:
        companies.update(json.load(f))


def drop_row(row):
    ticker = row['symbol']
    company_list = companies.get(ticker, [])

    def fmt(s: str) -> str:
        return s.lower().replace('.', '').replace(',', '').replace('\'', '').replace('-', ' ').replace(' ', '')
    # Ensure the first several chars of the company name is in the list
    title: str = fmt(row['title'])
    security = None
    if type(row['Security']) != float:
        security = fmt(row['Security'])
    length = int(max(len(title)/2, 3))
    if not any(fmt(i).startswith(title[:length]) for i in company_list) and \
            not (True if not security else title.startswith(security[:length])):
        return True
    else:
        return False

df = df[~df.apply(drop_row, axis=1)]

%xdel companies

print('Filings with Correct Name:', df.shape)
# Get the number of companies in each cap
print('\nCompany Counts:\n', df.groupby(
    'cap', observed=True)['symbol'].nunique())
# Get the number of companies in each cap by sector
print('\nCompany Counts by cap and sector:\n',
      df.groupby(['cap', 'GICS Sector'], observed=True)['symbol'].nunique())
# Get the number of filings in each cap by sector
print('\nFiling Counts by cap and sector:\n',
      df.groupby(['cap', 'GICS Sector'], observed=True).size())

Filings in Range: (53434, 353)
Filings with Correct Name: (53151, 353)

Company Counts:
 cap
large    498
mid      397
small    599
Name: symbol, dtype: int64

Company Counts by cap and sector:
 cap    GICS Sector           
large  Communication Services     22
       Consumer Discretionary     53
       Consumer Staples           37
       Energy                     22
       Financials                 70
       Health Care                64
       Industrials                78
       Information Technology     64
       Materials                  27
       Real Estate                31
       Utilities                  30
mid    Communication Services      8
       Consumer Discretionary     65
       Consumer Staples           18
       Energy                     20
       Financials                 64
       Health Care                35
       Industrials                73
       Information Technology     41
       Materials                  27
       Real Estate                3

### Derive Features

#### Earnings - BMO or AMC (Before Market Open or After Market Clore)


In [11]:
# Determine BMO or AMC
# We can assume any earnings date after 4pm is AMC, and before is BMO
def bmo_amc(date: pd.Timestamp) -> str:
    if date.hour >= 16:
        return 'AMC'
    else:
        return 'BMO'


df['BMO/AMC'] = df['Earnings Date'].apply(bmo_amc)
df['BMO/AMC'] = df['BMO/AMC'].astype('category')
df.tail(3)

,symbol,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccruedInterestReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AdjustedGeographySegmentData,...,GICS Sub-Industry,GICS Sector ID,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,title,Earnings Date,BMO/AMC
159838,META,2023-12-31,3M,USD,4.849000e+09,1.616900e+10,NaN,-3.313400e+10,7.325300e+10,NaN,...,Interactive Media & Services,50,5020,Media & Entertainment,502030.0,Interactive Media & Services,50203010.0,Meta Platforms,2024-02-01 16:05:00,AMC
159839,LOPE,2023-12-31,3M,USD,1.767600e+07,7.881100e+07,NaN,-1.704150e+08,3.225120e+08,NaN,...,Education Services,25,2530,Consumer Services,253020.0,Diversified Consumer Services,25302010.0,Grand Canyon Education,2024-02-13 16:05:00,AMC
159840,CRS,2023-12-31,3M,USD,3.134000e+08,5.084000e+08,NaN,NaN,3.279000e+08,NaN,...,Steel,15,1510,Materials,151040.0,Metals & Mining,15104050.0,Carpenter Tech,2024-01-25 08:00:00,BMO


#### Earnings - Effect on Stock Date


In [12]:
from pandas.tseries.holiday import nearest_workday, AbstractHolidayCalendar, Holiday, USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, USLaborDay, USThanksgivingDay
from pandas.tseries.offsets import CustomBusinessDay


class USStockMarketCalendar(AbstractHolidayCalendar):
    # Holidays for US Stock Market

    rules = [
        # New Year's Day
        Holiday('New Years', month=1, day=1, observance=nearest_workday),
        # Martin Luther King Jr. Day (3rd Monday in January)
        USMartinLutherKingJr,
        # Presidents' Day (3rd Monday in February)
        USPresidentsDay,
        # Good Friday (Friday before Easter Sunday)
        GoodFriday,
        # Memorial Day (last Monday in May)
        USMemorialDay,
        # Independence Day
        Holiday('Independence Day', month=7,
                day=4, observance=nearest_workday),
        # Labor Day (1st Monday in September)
        USLaborDay,
        # Thanksgiving Day (4th Thursday in November)
        USThanksgivingDay,
        # Christmas Day
        Holiday('Christmas', month=12, day=25, observance=nearest_workday),
        # Juneteenth (June 19th, observed starting 2022)
        Holiday('Juneteenth', month=6, day=19,
                start_date='2022-06-18', observance=nearest_workday),
    ]


# Create a custom business day offset using the custom calendar
bday = CustomBusinessDay(calendar=USStockMarketCalendar())


# Get the date the earnings call will affect the stock price
# If the earnings call is BMO, the stock price will be affected the same day (assuming earnings was on a weekday)
# If the earnings call is AMC, the stock price will be affected the next business day
def stock_effect_date(row: pd.Series):
    if row['BMO/AMC'] == 'BMO' and row['Earnings Date'].dayofweek < 5:
        return row['Earnings Date'].date()
    else:
        return (row['Earnings Date'] + bday).date()


df['Stock Affect Date'] = df.apply(stock_effect_date, axis=1)
display(df.tail(3))

,symbol,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccruedInterestReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AdjustedGeographySegmentData,...,GICS Sector ID,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,title,Earnings Date,BMO/AMC,Stock Affect Date
159838,META,2023-12-31,3M,USD,4.849000e+09,1.616900e+10,NaN,-3.313400e+10,7.325300e+10,NaN,...,50,5020,Media & Entertainment,502030.0,Interactive Media & Services,50203010.0,Meta Platforms,2024-02-01 16:05:00,AMC,2024-02-02
159839,LOPE,2023-12-31,3M,USD,1.767600e+07,7.881100e+07,NaN,-1.704150e+08,3.225120e+08,NaN,...,25,2530,Consumer Services,253020.0,Diversified Consumer Services,25302010.0,Grand Canyon Education,2024-02-13 16:05:00,AMC,2024-02-14
159840,CRS,2023-12-31,3M,USD,3.134000e+08,5.084000e+08,NaN,NaN,3.279000e+08,NaN,...,15,1510,Materials,151040.0,Metals & Mining,15104050.0,Carpenter Tech,2024-01-25 08:00:00,BMO,2024-01-25


#### Ratios


In [13]:
def column_nulls(df: pd.DataFrame):
    # Find columns within financials
    income_cols = ['revenue', 'sales', 'goodssold',
                   'profit', 'expense', 'income', 'ebit', 'eps', 'minority', 'dividend', 'research', 'r&d', 'provision']
    balance_cols = ['asset', 'liabilit', 'equity', 'debt', 'inventory', 'account',
                    'minority', 'share', 'current', 'accrued', 'development' 'reserve']
    cash_cols = ['flow', 'cash', 'capex']

    income_nulls = {}
    balance_nulls = {}
    cash_nulls = {}
    for col in df.columns:
        # Check for income-related columns
        if any(keyword in col.lower() for keyword in income_cols):
            null_count = df[col].isnull().sum()
            income_nulls[col] = null_count

        # Check for balance-related columns
        if any(keyword in col.lower() for keyword in balance_cols):
            null_count = df[col].isnull().sum()
            balance_nulls[col] = null_count

        # Check for cash-related columns
        if any(keyword in col.lower() for keyword in cash_cols):
            null_count = df[col].isnull().sum()
            cash_nulls[col] = null_count

    return income_nulls, balance_nulls, cash_nulls


income_nulls, balance_nulls, cash_nulls = column_nulls(df)
[display(pd.Series(x).sort_index().sort_values().head(60))
 for x in [income_nulls, balance_nulls, cash_nulls]]

%reset_selective -f income_nulls balance_nulls cash_nulls

NetIncomeCommonStockholders                               42
PretaxIncome                                              42
OperatingRevenue                                          42
NormalizedIncome                                          42
NetIncomeIncludingNoncontrollingInterests                 42
NetIncomeFromContinuingOperationNetMinorityInterest       42
NetIncomeFromContinuingAndDiscontinuedOperation           42
NetIncomeContinuousOperations                             42
NetIncome                                                 42
TotalRevenue                                              42
TotalLiabilitiesNetMinorityInterest                       74
TotalEquityGrossMinorityInterest                          74
NetIncomeFromContinuingOperations                        875
BasicEPS                                                 888
DilutedEPS                                               888
TaxProvision                                            1780
NetInterestIncome       

NetIncomeFromContinuingOperationNetMinorityInterest       42
TotalLiabilitiesNetMinorityInterest                       74
TotalEquityGrossMinorityInterest                          74
TotalAssets                                               74
StockholdersEquity                                        74
NetTangibleAssets                                         74
CommonStockEquity                                         74
ShareIssued                                               80
OrdinarySharesNumber                                      80
BasicAverageShares                                       882
DilutedAverageShares                                     882
PayablesAndAccruedExpenses                              2164
TotalDebt                                               2778
LongTermDebtAndCapitalLeaseObligation                   3394
AccountsReceivable                                      3608
NetIssuancePaymentsOfDebt                               4608
AccountsPayable         

FreeCashFlow                                       65
EndCashPosition                                    71
FinancingCashFlow                                  75
BeginningCashPosition                              84
ChangeInCashSupplementalAsReported                 90
InvestingCashFlow                                  94
OperatingCashFlow                                 117
CashAndCashEquivalents                            173
ChangesInCash                                     176
CashFlowFromContinuingFinancingActivities         208
CashFlowFromContinuingInvestingActivities         227
CashFlowFromContinuingOperatingActivities         924
CashCashEquivalentsAndShortTermInvestments       5313
OtherNonCashItems                                8449
CashDividendsPaid                               15288
AmortizationCashFlow                            40601
RestrictedCash                                  42091
CashFinancial                                   46306
CashFromDiscontinuedOperatin

In [14]:
# Columns with no null values
print('Columns with no null values:', len(
    df.columns[df.notnull().all()]), df.columns[df.notnull().all()])

Columns with no null values: 17 Index(['symbol', 'asOfDate', 'periodType', 'currencyCode', 'cap',
       'GICS Sector', 'GICS Sub-Industry', 'GICS Sector ID',
       'GICS Industry Group ID', 'GICS Industry Group', 'GICS Industry ID',
       'GICS Industry', 'GICS Sub-Industry ID', 'title', 'Earnings Date',
       'BMO/AMC', 'Stock Affect Date'],
      dtype='object')


In [15]:
cols = ['TotalRevenue', 'OperatingRevenue', 'NetIncome', 'BasicEPS', 'DilutedEPS', 'EBITDA', 'EBIT']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Nulls:\n', df.dropna(subset=cols + ['CurrentLiabilities']).isnull().sum().sort_values())

Nulls:
 symbol                                                              0
PretaxIncome                                                        0
OperatingRevenue                                                    0
OperatingIncome                                                     0
NormalizedIncome                                                    0
NormalizedEBITDA                                                    0
NetTangibleAssets                                                   0
NetIncomeIncludingNoncontrollingInterests                           0
NetIncomeFromContinuingOperationNetMinorityInterest                 0
NetIncomeFromContinuingAndDiscontinuedOperation                     0
NetIncomeContinuousOperations                                       0
NetIncomeCommonStockholders                                         0
NetIncome                                                           0
BMO/AMC                                                             0
EBITDA      

##### Liquidity Ratios


In [16]:
cols += ['Current Ratio', 'Quick Ratio']

# Current Ratio
required_cols = ['Current Ratio']
df['Current Ratio'] = df['CurrentAssets'] / df['CurrentLiabilities']

# Acid Test Ratio
# Not all companies report Inventory
df['Quick Ratio'] = (df['CurrentAssets'] - df['Inventory']) / \
    df['CurrentLiabilities']

##### Profitability Ratios


In [17]:
# Gross Profit Margin
required_cols += ['Gross Profit Margin']
cols += ['Gross Profit Margin']
df['Gross Profit Margin'] = df['GrossProfit'] / df['TotalRevenue']

# Net Profit Margin
required_cols += ['Net Profit Margin']
cols.append('Net Profit Margin')
df['Net Profit Margin'] = df['NetIncome'] / df['TotalRevenue']

# Cash Flow Margin
required_cols += ['Cash Flow Margin']
cols += ['Cash Flow Margin']
df['Cash Flow Margin'] = df['OperatingCashFlow'] / df['TotalRevenue']

# Operating Margin
required_cols += ['Operating Margin']
cols.append('Operating Margin')
df['Operating Margin'] = df['OperatingIncome'] / df['TotalRevenue']

# # Net Interest Margin
# # Not relevant for all companies
# cols += ['Net Interest Margin']
# df['Net Interest Margin'] = df['NetInterestIncome'] / df['TotalRevenue']

# Return on Assets (ROA)
required_cols += ['ROA']
cols += ['ROA']
df['ROA'] = df['NetIncome'] / df['TotalAssets']

# Cash Return on Assets (CROA)
required_cols += ['CROA']
cols.append('CROA')
df['CROA'] = df['OperatingCashFlow'] / df['TotalAssets']

# Return on Equity (ROE)
required_cols += ['ROE']
cols += ['ROE']
df['ROE'] = df['NetIncome'] / df['TotalEquityGrossMinorityInterest']

##### Efficiency Ratios


In [18]:
# Efficiency Ratio (Expense as a % of Revenue)
# Not all filings report OperatingExpense and InterestExpense
cols.append('Efficiency Ratio')
df['Efficiency Ratio'] = (df['TotalExpenses'] - df['InterestExpense']) / df['TotalRevenue']

# Inventory Turnover
# Not all companies report ChangeInInventory
cols.append('Inventory Turnover')
df['Inventory Turnover'] = df['CostOfRevenue'] / df['ChangeInInventory']

##### Debt Ratios


In [19]:
cols += ['Debt to Equity', 'Debt Ratio', 'Interest Coverage']

# Debt to Equity
required_cols += ['Debt to Equity']
df['Debt to Equity'] = df['TotalDebt'] / \
    df['TotalEquityGrossMinorityInterest']

# Debt Ratio
required_cols += ['Debt Ratio']
df['Debt Ratio'] = df['TotalDebt'] / df['TotalAssets']

# Interest Coverage
# Not all companies report InterestExpense
df['Interest Coverage'] = df['EBIT'] / df['InterestExpense']

# Cash Flow to Debt
required_cols += ['Cash Flow to Debt']
cols.append('Cash Flow to Debt')
df['Cash Flow to Debt'] = df['OperatingCashFlow'] / df['TotalDebt']

#### Other


In [20]:
# Assets to Equity
required_cols += ['Assets to Equity']
cols.append('Assets to Equity')
df['Assets to Equity'] = df['TotalAssets'] / df['TotalEquityGrossMinorityInterest']

# R&D to Revenue
# Not relevant for all companies
cols += ['R&D to Revenue']
df['R&D to Revenue'] = df['ResearchAndDevelopment'] / df['TotalRevenue']

# Investment Cash Flow to OCF
required_cols += ['Investment CF to OCF']
cols.append('Investment CF to OCF')
df['Investment CF to OCF'] = df['OperatingCashFlow'] / df['InvestingCashFlow']

# Financing Cash Flow to OCF
required_cols += ['Financing CF to OCF']
cols.append('Financing CF to OCF')
df['Financing CF to OCF'] = df['FinancingCashFlow'] / df['OperatingCashFlow']

# FCF
required_cols += ['FreeCashFlow']
cols.append('FreeCashFlow')

# Capex
required_cols += ['Capex']
cols.append('Capex')
df['Capex'] = df['OperatingCashFlow'] - df['FreeCashFlow']

# Book Value Per Share
required_cols += ['Book Value Per Share']
cols += ['Book Value Per Share']
df['Book Value Per Share'] = df['TotalEquityGrossMinorityInterest'] / \
    df['OrdinarySharesNumber']

# Tangible Book Value Per Share
required_cols += ['Tangible Book Value Per Share']
cols += ['Tangible Book Value Per Share']
df['Tangible Book Value Per Share'] = df['TangibleBookValue'] / \
    df['OrdinarySharesNumber']

# Any infs are from division by zero, we'll replace them with 0
df.replace([np.inf, -np.inf], 0, inplace=True)

# Total filings
print('Total Filings:', df.shape)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # Print nulls
    print('\nNulls:\n', df.dropna(subset=required_cols).isnull().sum().sort_values())

Total Filings: (53151, 377)

Nulls:
 symbol                                                              0
TotalDebt                                                           0
TotalCapitalization                                                 0
TotalAssets                                                         0
TaxRateForCalcs                                                     0
TaxEffectOfUnusualItems                                             0
TangibleBookValue                                                   0
StockholdersEquity                                                  0
ShareIssued                                                         0
ReconciledCostOfRevenue                                             0
PretaxIncome                                                        0
OrdinarySharesNumber                                                0
OperatingRevenue                                                    0
OperatingIncome                                      

#### Percent Change


In [21]:
# Re-sort
def sort_cols(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    return df[cols]

df = sort_cols(df, cat_cols + ['asOfDate', 'Stock Affect Date'] + [col for col in cols])
df = df.sort_values(['symbol', 'Stock Affect Date']).reset_index(drop=True)

pd.set_option('display.max_columns', None)

In [22]:
display(df.query('symbol == "UNFI"').tail(30))

# Calculate the difference in quarters between current and previous report for the same company
df['quarter_diff'] = (df.groupby('symbol', observed=True)['asOfDate'].diff() / pd.Timedelta(90, 'days')).round()

# Identify where the streak of consecutive quarters breaks
df['streak_id'] = (df['quarter_diff'] != 1).cumsum()

# Group by each streak and calculate the percentage change within the streak
# Note: Since we are dealing with potentially negative numbers, we'll use the absolute value in the denominator
# not sure why pandas default pct_change method doesn't use this formula or at least have it as an option
df[cols] = df.groupby(['symbol', 'streak_id'], observed=True)[
    cols].diff() / abs(df.groupby(['symbol', 'streak_id'], observed=True)[cols].shift())

# Drop streak ID and quarter diff and asOfDate
df.drop(columns=['streak_id', 'quarter_diff', 'asOfDate'], inplace=True)

# Any infinities are from division by zero, we'll replace them with 0
df.replace([np.inf, -np.inf], 0, inplace=True)

print('Percent Change')
display(df.query('symbol == "UNFI"').tail(30))

# Print Nulls
print(df.shape)
print(df.isnull().sum().sort_values())

,symbol,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,asOfDate,Stock Affect Date,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share
48662,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-01-31,2015-03-10,2.016546e+09,2.016546e+09,27844000.0,0.56,0.55,65076000.0,49577000.0,2.684283,1.092395,0.148372,0.013808,0.022734,0.024671,0.011089,0.018258,0.021326,0.973566,29.489440,0.425084,0.221038,13.949634,0.082602,1.923131,NaN,-1.625443,-0.156945,17053000.0,28791000.0,26.075493,18.135368
48663,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-04-30,2015-06-09,2.114643e+09,2.114643e+09,41750000.0,0.83,0.83,91040000.0,73491000.0,2.733705,1.089708,0.154122,0.019743,0.022087,0.032616,0.016319,0.018257,0.030857,0.965530,-74.012289,0.406060,0.214755,18.747704,0.085011,1.890812,NaN,-1.103769,-0.182075,4325000.0,42381000.0,27.013556,19.092639
48664,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-07-31,2015-09-16,2.061313e+09,2.061313e+09,36098000.0,0.72,0.72,79923000.0,63329000.0,2.929077,1.081909,0.153497,0.017512,0.026710,0.031576,0.014206,0.021668,0.026137,0.966595,-47.149454,0.396466,0.215489,16.802600,0.100551,1.839849,NaN,-1.646244,-0.510980,24467000.0,30590000.0,27.568828,19.734470
48665,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-10-31,2015-12-08,2.076649e+09,2.076649e+09,30131000.0,0.60,0.60,70587000.0,53883000.0,2.563861,0.890808,0.151175,0.014509,0.002645,0.027310,0.011369,0.002072,0.021212,0.970885,-17.559166,0.351689,0.188501,14.376467,0.010994,1.865714,NaN,-0.722156,-0.544064,-2096000.0,7588000.0,28.231482,20.467395
48666,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-01-31,2016-03-08,2.047712e+09,2.047712e+09,22683000.0,0.45,0.45,57487000.0,41344000.0,2.958850,1.105978,0.145293,0.011077,0.058172,0.021335,0.009121,0.047896,0.015736,0.976906,12.544665,0.316604,0.183501,11.478068,0.261014,1.725351,NaN,-9.276458,-0.889640,106235000.0,12884000.0,28.639526,20.944130
48667,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-04-30,2016-06-07,2.132104e+09,2.132104e+09,38271000.0,0.76,0.76,85180000.0,67060000.0,2.592140,0.959339,0.151228,0.017950,0.037994,0.030963,0.014464,0.030616,0.025678,0.966981,-47.733462,0.298555,0.168172,15.296533,0.182050,1.775294,NaN,-0.828962,0.298875,72406000.0,8601000.0,29.583545,20.055101
48668,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-07-31,2016-09-13,2.213821e+09,2.213821e+09,34681000.0,0.69,0.69,82233000.0,62194000.0,2.599859,0.951277,0.156048,0.015666,0.041101,0.028568,0.012160,0.031903,0.022824,0.969388,2087.550838,0.394939,0.210407,13.744530,0.151623,1.877030,NaN,-0.390942,1.550659,78689000.0,12302000.0,30.159062,18.478892
48669,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-10-31,2016-12-08,2.278364e+09,2.278364e+09,29217000.0

Percent Change


,symbol,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,Stock Affect Date,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share
48662,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-03-10,0.012080,0.012080,-0.157315,-0.151515,-0.166667,-0.096518,-0.143304,0.043840,0.136145,-0.073256,-0.167373,1.458734,-0.166093,-0.157591,1.464124,-0.173919,0.004939,3.656654,-0.006245,0.013396,-0.215378,1.457989,-0.019381,NaN,-1.627276,0.887911,1.135218,0.051841,0.018612,0.038344
48663,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-06-09,0.048646,0.048646,0.499425,0.482143,0.509091,0.398980,0.482361,0.018412,-0.002459,0.038757,0.429868,-0.028459,0.322032,0.471626,-0.000086,0.446894,-0.008255,-3.509790,-0.044753,-0.028425,0.343957,0.029169,-0.016806,NaN,0.320943,-0.160118,-0.746379,0.472023,0.035975,0.052785
48664,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-09-16,-0.025219,-0.025219,-0.135377,-0.132530,-0.132530,-0.122111,-0.138275,0.071468,-0.007157,-0.004056,-0.113008,0.209297,-0.031883,-0.129481,0.186838,-0.152944,0.001103,0.362951,-0.023627,0.003418,-0.103752,0.182795,-0.026953,NaN,-0.491475,-1.806422,4.657110,-0.278214,0.020555,0.033617
48665,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2015-12-08,0.007440,0.007440,-0.165300,-0.166667,-0.166667,-0.116812,-0.149158,-0.124687,-0.176633,-0.015131,-0.171464,-0.900985,-0.135120,-0.199691,-0.904359,-0.188440,0.004438,0.627585,-0.112941,-0.125238,-0.144390,-0.890666,0.014058,NaN,0.561331,-0.064747,-1.085666,-0.751945,0.024036,0.037139
48666,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-03-08,-0.013934,-0.013934,-0.247187,-0.250000,-0.250000,-0.185587,-0.232708,0.154060,0.241545,-0.038908,-0.236549,20.996052,-0.218779,-0.197797,22.112545,-0.258149,0.006201,1.714423,-0.099762,-0.026524,-0.201607,22.742297,-0.075233,NaN,-11.845496,-0.635175,51.684637,0.697944,0.014454,0.023292
48667,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-06-07,0.041213,0.041213,0.687211,0.688889,0.688889,0.481726,0.622001,-0.123936,-0.132588,0.040847,0.620428,-0.346866,0.451268,0.585877,-0.360793,0.631783,-0.010160,-4.805081,-0.057009,-0.083537,0.332675,-0.302528,0.028946,NaN,0.910638,1.335951,-0.318436,-0.332428,0.032962,-0.042448
48668,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-09-13,0.038327,0.038327,-0.093805,-0.092105,-0.092105,-0.034597,-0.072562,0.002978,-0.008404,0.031877,-0.127254,0.081787,-0.077352,-0.159327,0.042033,-0.111150,0.002489,44.733489,0.322836,0.251138,-0.101461,-0.167132,0.057307,NaN,0.528396,4.188312,0.086775,0.430299,0.019454,-0.078594
48669,UNFI,small,30,Consumer Staples,3010,Consumer Staples Distribution & Retail,301010.0,Consumer Staples Distribution & Retail,30101030.0,Food Retail,2016-12-08,0.029155,0.029155,-0.157550,-0.159420,-0.159420,-0.096835,-0.146943,-0.040097,-0.035577,-0.018335,-0.181416,-1.077474,-0.180508,-0.183

(53151, 41)
symbol                               0
cap                                  0
GICS Sector ID                       0
GICS Sector                          0
GICS Industry Group ID               0
GICS Industry Group                  0
GICS Industry ID                     0
GICS Industry                        0
GICS Sub-Industry ID                 0
GICS Sub-Industry                    0
Stock Affect Date                    0
NetIncome                         5590
FreeCashFlow                      5612
Assets to Equity                  5617
Book Value Per Share              5618
Tangible Book Value Per Share     5618
TotalRevenue                      5621
OperatingRevenue                  5621
Net Profit Margin                 5621
ROE                               5630
ROA                               5630
Cash Flow Margin                  5688
Investment CF to OCF              5690
Financing CF to OCF               5692
CROA                              5692
BasicEPS     

In [23]:
# Mark days with an earnings call
df['Earnings Call'] = 1

# Get valid tickers and reset categories
valid_tickers = df['symbol'].unique()
df['symbol'] = df['symbol'].cat.set_categories(valid_tickers)

##### Stock Price


In [24]:
# Grab stock prices
df_stock = pd.concat([pd.read_csv(f'data/{cap}/stock_hist.csv')[['symbol', 'date', 'adjclose']]
                      for cap in ['large', 'mid', 'small']])
cols = ['date', 'adjclose'] + cols
df_stock['date'] = df_stock['date'].str[:10]

# Valid tickers
df_stock = df_stock[df_stock['symbol'].isin(valid_tickers)]
df_stock['symbol'] = df_stock['symbol'].astype(df['symbol'].dtype)

df['Stock Affect Date'] = df['Stock Affect Date'].astype('datetime64[ns]')

df_stock['Stock Affect Date'] = df_stock['date']
df_stock['Stock Affect Date'] = df_stock['Stock Affect Date'].astype('datetime64[ns]')

# Merge stock prices with financials
df = pd.merge_asof(df_stock.sort_values('Stock Affect Date'), df.sort_values('Stock Affect Date'), on='Stock Affect Date', by='symbol', tolerance=pd.Timedelta(days=0))
%xdel df_stock

print('Total:', df.shape)
display(df.head(3))
# Drop invalid tickers
df = df[df['symbol'].isin(valid_tickers)]

Total: (3715142, 44)


,symbol,date,adjclose,Stock Affect Date,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,Earnings Call
0,MMM,2013-07-29,81.858749,2013-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ON,2013-07-29,8.100000,2013-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OKE,2013-07-29,23.889841,2013-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Fill cat_cols
group = df.groupby('symbol', observed=True)
for col in cat_cols:
    df[col] = group[col].ffill()
    df[col] = group[col].bfill()
%xdel group
    
# Drop any remaining nulls in cat_cols
df = df.dropna(subset=cat_cols)

print(df.shape)
print(df.isnull().sum())

(3715142, 44)
symbol                                 0
date                                   0
adjclose                               0
Stock Affect Date                      0
cap                                    0
GICS Sector ID                         0
GICS Sector                            0
GICS Industry Group ID                 0
GICS Industry Group                    0
GICS Industry ID                       0
GICS Industry                          0
GICS Sub-Industry ID                   0
GICS Sub-Industry                      0
TotalRevenue                     3667844
OperatingRevenue                 3667844
NetIncome                        3667815
BasicEPS                         3669356
DilutedEPS                       3669356
EBITDA                           3674869
EBIT                             3672965
Current Ratio                    3674891
Quick Ratio                      3685643
Gross Profit Margin              3675253
Net Profit Margin                3667844
Ca

##### Macros


In [26]:
def get_macro(path: str, freq: str) -> pd.DataFrame:
    # All are forward filled as data is not published on federal holidays, despite some being trading days
    df = pd.read_csv(f'data/macro/{path}.csv')
    if freq == 'weekly':
        df = df[['realtime_start', 'date', 'value']]
        df.columns = ['date', 'week', f'{path}']
        df['date'] = df['date'].astype('datetime64[ns]')
        df['week'] = df['week'].astype('datetime64[ns]')
        df.set_index('date', inplace=True)
        # Only keep when date published is the same
        df = df[df.index == df['week']]
        df.drop(columns=['week'], inplace=True)
        # Ensure our weeklies line up with all trading dates
        df = df.asfreq(bday, method='ffill') 
        return df

    elif freq == 'monthly':
        df = df[['realtime_start', 'date', 'value']]
        df.columns = ['date', 'month', f'{path}']
        df['date'] = df['date'].astype('datetime64[ns]')
        df['month'] = df['month'].astype('datetime64[ns]')
        df.set_index('date', inplace=True)
        # Only keep when date published is in the following month
        df = df[df.index <= df['month'] + pd.DateOffset(months=2)]
        df.drop(columns=['month'], inplace=True)
        # Forward fill
        df.ffill(inplace=True)
        return df.diff() / abs(df.shift())

    elif freq == 'quarterly':
        df = df[['realtime_start', 'date', 'value']]
        df.columns = ['date', 'quarter', f'{path}']
        df['date'] = df['date'].astype('datetime64[ns]')
        df['quarter'] = df['quarter'].astype('datetime64[ns]')
        df.set_index('date', inplace=True)
        # Only keep when date published is in the following quarter
        df = df[df.index <= df['quarter'] + pd.DateOffset(months=4)]
        df.drop(columns=['quarter'], inplace=True)
        # Forward fill
        df.ffill(inplace=True)
        return df.diff() / abs(df.shift())

    # We need some extra work for the pct_change later for the dailies
    elif freq == 'daily':
        df.columns = ['date', f'{path}']
        df['date'] = df['date'].astype('datetime64[ns]')
        df.set_index('date', inplace=True)
        # Ensure our dailies line up with all trading dates
        df = df.asfreq(bday, method='ffill')
        return df


# Dailies
# These are going to need some extra work for percent change after resampling to monthly
dailies = ['fed_funds', 'treasury_10yr', 'treasury_2yr',
           'baa_credit_spread', 'aaa_credit_spread']
cols_need_pchange = dailies.copy()
cols += dailies

fed_funds, treasury_10yr, treasury_2yr, baa_credit_spread, aaa_credit_spread = [
    get_macro(path, 'daily') for path in dailies]


def get_index(path: str) -> pd.DataFrame:
    df = pd.read_csv(f'data/macro/{path}.csv')[['date', 'adjclose']]
    df.columns = ['date', path]
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    # Ensure all my indexes line up with all trading dates 
    # (ex. ^IRX is closed on some federal holidays that are trading days)
    df = df.asfreq(bday, method='ffill')
    return df


dailies_index = ['treasury_3mo', 'vix', 'gold', 'oil', 'silver', 'copper', 'aluminium', 'platinum', 'palladium', 'natural_gas', 'wheat', 'corn', 'soybean', 'coffee', 'cocao', 'sugar', 'cotton']
cols_need_pchange += dailies_index
cols += dailies_index
treasury_3mo, vix, gold, oil, silver, copper, aluminium, platinum, palladium, natural_gas, wheat, corn, soybean, coffee, cocao, sugar, cotton = [get_index(path) for path in dailies_index]
dailies += dailies_index

# Weeklies
weeklies = ['mortgage_30yr', 'mortgage_15yr']
cols_need_pchange += weeklies
cols += weeklies
mortgage_30yr, mortgage_15yr = [
    get_macro(path, 'weekly') for path in weeklies]

# Monthlies
monthlies = ['cpi', 'unemployment', 'federal_budget_deficit', 'housing_starts']
cols += monthlies

cpi, unemployment, federal_budget_deficit, housing_starts = [
    get_macro(path, 'monthly') for path in monthlies]

# Quarterlies
quarterlies = ['gdp_real', 'avg_home_price']
cols += quarterlies
gdp_real, avg_home_price = [get_macro(path, 'quarterly') for path in quarterlies]

# Any infs are from division by zero, we'll replace them with 0
for var in dailies + weeklies + monthlies + quarterlies:
    globals()[var].replace([np.inf, -np.inf], 0, inplace=True)

print(cols_need_pchange, dailies, weeklies, monthlies, quarterlies, sep='\n')

['fed_funds', 'treasury_10yr', 'treasury_2yr', 'baa_credit_spread', 'aaa_credit_spread', 'treasury_3mo', 'vix', 'gold', 'oil', 'silver', 'copper', 'aluminium', 'platinum', 'palladium', 'natural_gas', 'wheat', 'corn', 'soybean', 'coffee', 'cocao', 'sugar', 'cotton', 'mortgage_30yr', 'mortgage_15yr']
['fed_funds', 'treasury_10yr', 'treasury_2yr', 'baa_credit_spread', 'aaa_credit_spread', 'treasury_3mo', 'vix', 'gold', 'oil', 'silver', 'copper', 'aluminium', 'platinum', 'palladium', 'natural_gas', 'wheat', 'corn', 'soybean', 'coffee', 'cocao', 'sugar', 'cotton']
['mortgage_30yr', 'mortgage_15yr']
['cpi', 'unemployment', 'federal_budget_deficit', 'housing_starts']
['gdp_real', 'avg_home_price']


In [27]:
# Merge macros with financials & stock prices
df['date'] = df['date'].astype('datetime64[ns]')
for name in dailies + weeklies + monthlies + quarterlies:
	var = globals()[name]
	df = pd.merge_asof(df.sort_values('date'), var.sort_values('date'), on='date', tolerance=pd.Timedelta(days=0))

%reset_selective -f fed_funds treasury_10yr treasury_2yr treasury_3mo baa_credit_spread aaa_credit_spread gold oil vix mortgage_30yr mortgage_15yr cpi unemployment federal_budget_deficit housing_starts gdp avg_home_price

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df.head(5))
	print('\nNulls:\n', df.isnull().sum().sort_values())

,symbol,date,adjclose,Stock Affect Date,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,Earnings Call,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
0,MMM,2013-07-29,81.858749,2013-07-29,large,20,Industrials,2010,Capital Goods,201050.0,Industrial Conglomerates,20105010.0,Industrial Conglomerates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
1,SLCA,2013-07-29,21.858809,2013-07-29,small,10,Energy,1010,Energy,101010.0,Energy Equipment & Services,10101020.0,Oil & Gas Equipment & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
2,XRAY,2013-07-29,38.631176,2013-07-29,large,35,Health Care,3510,Health Care Equipment & Services,351010.0,Health Care Equipment & Supplies,35101020.0,Health Care Supplies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
3,FE,2013-07-29,24.726622,2013-07-29,large,55,Utilities,5510,Utilities,551010.0,Electric Utilities,55101010.0,Electric Utilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
4,LLY,2013-07-29,42.245495,2013-07-29,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352020.0,Pharmaceuticals,35202010.0,Pharmaceuticals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN



Nulls:
 symbol                                 0
GICS Sub-Industry                      0
GICS Industry                          0
GICS Industry ID                       0
GICS Industry Group                    0
GICS Industry Group ID                 0
GICS Sub-Industry ID                   0
GICS Sector ID                         0
cap                                    0
Stock Affect Date                      0
adjclose                               0
date                                   0
GICS Sector                            0
natural_gas                         1475
palladium                           1475
platinum                            1475
copper                              1475
gold                                1475
oil                                 1475
vix                                 1475
treasury_3mo                        1475
wheat                               1475
silver                              1475
corn                                1475
cocao  

##### Resample


In [28]:
print(df.shape)
display(df.head(3))

# Resample to monthly
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df = df.groupby('symbol', observed=True).resample(
    'BME', include_groups=False, fill_method='ffill').last()

print(df.shape)
print('\nResampled to Monthly:')

# Reset index and sort
df.reset_index(inplace=True)
df.sort_values(['symbol', 'date'], inplace=True)

display(df.head(5))

# Apply percent change
print('\nApply Precent Change')
# Adjusted close
df['adjclose'] = df.groupby('symbol', observed=True)['adjclose'].diff() / abs(df.groupby('symbol', observed=True)['adjclose'].shift())
# Weeklies and dailies
for col in cols_need_pchange:
    df[col] = df.groupby('symbol', observed=True)[col].diff() / abs(df.groupby('symbol', observed=True)[col].shift())

display(df.head(5))

# Any infs are from division by zero, we'll replace them with 0
df.replace([np.inf, -np.inf], 0, inplace=True)

# Drop where an earnings call occurred and required columns are null
df = df[~((df['Earnings Call'] == 1) & df[required_cols].isnull().any(axis=1))]
# Drop Earnings call column
df.drop(columns=['Earnings Call'], inplace=True)

# Sort cols
df = sort_cols(df, cat_cols + [col for col in cols])

print(df.shape)
display(df.head(3))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('\nNulls:\n', df.isnull().sum().sort_values())

(3715142, 74)


,symbol,date,adjclose,Stock Affect Date,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,Earnings Call,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
0,MMM,2013-07-29,81.858749,2013-07-29,large,20,Industrials,2010,Capital Goods,201050.0,Industrial Conglomerates,20105010.0,Industrial Conglomerates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
1,SLCA,2013-07-29,21.858809,2013-07-29,small,10,Energy,1010,Energy,101010.0,Energy Equipment & Services,10101020.0,Oil & Gas Equipment & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN
2,XRAY,2013-07-29,38.631176,2013-07-29,large,35,Health Care,3510,Health Care Equipment & Services,351010.0,Health Care Equipment & Supplies,35101020.0,Health Care Supplies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,2.61,0.33,2.69,1.76,0.02,13.39,1328.400024,104.550003,19.854,3.1165,NaN,1441.800049,743.75,3.459,651.5,489.25,1367.5,121.150002,2280.0,16.92,84.849998,4.31,3.39,NaN,NaN,NaN,NaN,NaN,NaN


(178944, 72)

Resampled to Monthly:


,symbol,date,adjclose,Stock Affect Date,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,Earnings Call,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
0,A,2013-07-31,29.255550,2013-07-31,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,2.60,0.31,2.68,1.78,0.030,13.45,1312.400024,105.029999,19.617001,3.1185,NaN,1428.400024,725.450012,3.446,664.25,499.00,1374.00,118.599998,2298.0,16.969999,85.629997,4.31,3.39,NaN,NaN,NaN,NaN,0.138079,NaN
1,A,2013-08-30,30.504780,2013-08-30,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.07,2.78,0.39,2.56,1.71,0.020,17.01,1396.099976,107.650002,23.462999,3.2250,NaN,1526.199951,722.099976,3.581,643.25,495.00,1424.00,112.099998,2413.0,16.340000,83.699997,4.51,3.54,0.001606,-0.026316,NaN,0.07177,NaN,NaN
2,A,2013-09-30,33.597343,2013-09-30,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,2.64,0.33,2.75,1.92,0.005,16.60,1326.500000,102.330002,21.656000,3.3210,NaN,1408.099976,726.150024,3.560,678.50,441.50,1282.75,113.699997,2640.0,17.480000,86.110001,4.32,3.37,0.000891,-0.013514,NaN,-0.00558,NaN,NaN
3,A,2013-10-31,33.276123,2013-10-31,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07,2.57,0.31,2.65,1.91,0.033,13.75,1323.599976,96.379997,21.832001,3.2945,NaN,1445.900024,736.250000,3.581,667.50,428.25,1280.25,105.400002,2677.0,18.320000,77.180000,4.10,3.20,0.001803,-0.013699,NaN,NaN,NaN,NaN
4,A,2013-11-29,35.118237,2013-11-29,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,0.039952,0.039952,0.255952,0.28,0.285714,0.148485,0.188285,0.051757,0.073614,0.019997,0.207703,0.686125,0.161234,0.207999,0.686539,0.137457,-0.028725,-0.499603,-0.095017,-0.038893,0.069456,0.754788,-0.058396,-0.02717,-0.856635,1.010306,1.12963,-0.396226,0.096854,0.551626,1.0,0.07,2.75,0.28,2.61,1.87,0.055,13.70,1250.599976,92.720001,19.981001,3.2305,NaN,1367.900024,718.000000,3.954,655.00,415.25,1336.50,110.250000,2791.0,17.150000,78.139999,4.29,3.30,-0.000594,0.013889,NaN,NaN,NaN,NaN



Apply Precent Change


,symbol,date,adjclose,Stock Affect Date,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,Earnings Call,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
0,A,2013-07-31,NaN,2013-07-31,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.138079,NaN
1,A,2013-08-30,0.042701,2013-08-30,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-0.222222,0.069231,0.258065,-0.044776,-0.039326,-0.333333,0.264684,0.063776,0.024945,0.196054,0.034151,NaN,0.068468,-0.004618,0.039176,-0.031615,-0.008016,0.036390,-0.054806,0.050044,-0.037124,-0.022539,0.046404,0.044248,0.001606,-0.026316,NaN,0.07177,NaN,NaN
2,A,2013-09-30,0.101380,2013-09-30,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.142857,-0.050360,-0.153846,0.074219,0.122807,-0.750000,-0.024103,-0.049853,-0.049419,-0.077015,0.029768,NaN,-0.077382,0.005609,-0.005864,0.054800,-0.108081,-0.099192,0.014273,0.094074,0.069767,0.028793,-0.042129,-0.048023,0.000891,-0.013514,NaN,-0.00558,NaN,NaN
3,A,2013-10-31,-0.009561,2013-10-31,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,-0.026515,-0.060606,-0.036364,-0.005208,5.600000,-0.171687,-0.002186,-0.058145,0.008127,-0.007980,NaN,0.026845,0.013909,0.005899,-0.016212,-0.030011,-0.001949,-0.072999,0.014015,0.048055,-0.103705,-0.050926,-0.050445,0.001803,-0.013699,NaN,NaN,NaN,NaN
4,A,2013-11-29,0.055358,2013-11-29,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,0.039952,0.039952,0.255952,0.28,0.285714,0.148485,0.188285,0.051757,0.073614,0.019997,0.207703,0.686125,0.161234,0.207999,0.686539,0.137457,-0.028725,-0.499603,-0.095017,-0.038893,0.069456,0.754788,-0.058396,-0.02717,-0.856635,1.010306,1.12963,-0.396226,0.096854,0.551626,1.0,0.000000,0.070039,-0.096774,-0.015094,-0.020942,0.666667,-0.003636,-0.055153,-0.037975,-0.084784,-0.019426,NaN,-0.053946,-0.024788,0.104161,-0.018727,-0.030356,0.043937,0.046015,0.042585,-0.063865,0.012438,0.046341,0.031250,-0.000594,0.013889,NaN,NaN,NaN,NaN


(162174, 72)


,symbol,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,date,adjclose,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
0,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2013-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.138079,NaN
2,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2013-09-30,0.101380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.142857,-0.050360,-0.153846,0.074219,0.122807,-0.75,-0.024103,-0.049853,-0.049419,-0.077015,0.029768,NaN,-0.077382,0.005609,-0.005864,0.054800,-0.108081,-0.099192,0.014273,0.094074,0.069767,0.028793,-0.042129,-0.048023,0.000891,-0.013514,NaN,-0.00558,NaN,NaN
3,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2013-10-31,-0.009561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,-0.026515,-0.060606,-0.036364,-0.005208,5.60,-0.171687,-0.002186,-0.058145,0.008127,-0.007980,NaN,0.026845,0.013909,0.005899,-0.016212,-0.030011,-0.001949,-0.072999,0.014015,0.048055,-0.103705,-0.050926,-0.050445,0.001803,-0.013699,NaN,NaN,NaN,NaN



Nulls:
 symbol                                0
cap                                   0
GICS Sector ID                        0
GICS Sector                           0
GICS Industry Group ID                0
GICS Industry Group                   0
GICS Industry ID                      0
GICS Industry                         0
GICS Sub-Industry ID                  0
GICS Sub-Industry                     0
date                                  0
palladium                          1317
platinum                           1317
copper                             1317
silver                             1317
oil                                1317
treasury_2yr                       1317
vix                                1317
treasury_3mo                       1317
aaa_credit_spread                  1317
baa_credit_spread                  1317
natural_gas                        1317
gold                               1317
wheat                              1317
cocao                          

In [29]:
# Drop dates outside of the date range
df = df[(df['date'] >= DATE_START) &
        (df['date'] <= DATE_END)]
print('In Range:', df.shape)

# Display null adjusted close dates
display(df[df['adjclose'].isnull()])

In Range: (154512, 72)


,symbol,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,date,adjclose,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
1408,ABNB,large,25,Consumer Discretionary,2530,Consumer Services,253010.0,"Hotels, Restaurants & Leisure",25301020.0,"Hotels, Resorts & Cruise Lines",2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001890,NaN,0.488626,0.011111,NaN,NaN
3175,ADNT,mid,25,Consumer Discretionary,2510,Automobiles & Components,251010.0,Automobile Components,25101010.0,Automotive Parts & Equipment,2016-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002900,NaN,NaN,-0.083187,0.007662,-0.007971
5056,AGTI,small,35,Health Care,3510,Health Care Equipment & Services,351020.0,Health Care Providers & Services,35102015.0,Health Care Services,2021-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016360,0.026189
5219,AHCO,small,35,Health Care,3510,Health Care Equipment & Services,351010.0,Health Care Equipment & Supplies,35101010.0,Health Care Equipment,2018-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5801,AIRC,mid,60,Real Estate,6010,Equity Real Estate Investment Trusts (REITs),601060.0,Residential REITs,60106010.0,Multi-Family Residential REITs,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011111,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176864,XHR,small,60,Real Estate,6010,Equity Real Estate Investment Trusts (REITs),601030.0,Hotel & Resort REITs,60103010.0,Hotel & Resort REITs,2015-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006233,0.017857,NaN,-0.022039,NaN,NaN
177224,XPEL,small,25,Consumer Discretionary,2510,Automobiles & Components,251010.0,Automobile Components,25101010.0,Automotive Parts & Equipment,2019-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [30]:
# Drop null adjusted close
df = df[df['adjclose'].notnull()]

# Replace inf with nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Get the number of companies in each cap
print('\nCompany Counts:\n', df.groupby(
    'cap', observed=True)['symbol'].nunique())

# Display nulls
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.isnull().sum().sort_values())

display(df.head(3))
df.to_csv('data/preprocessed.csv', index=False)


Company Counts:
 cap
large    498
mid      396
small    598
Name: symbol, dtype: int64


symbol                                0
coffee                                0
soybean                               0
corn                                  0
wheat                                 0
natural_gas                           0
palladium                             0
platinum                              0
treasury_10yr                         0
copper                                0
oil                                   0
gold                                  0
vix                                   0
treasury_3mo                          0
aaa_credit_spread                     0
baa_credit_spread                     0
treasury_2yr                          0
silver                                0
sugar                                 0
cocao                                 0
mortgage_30yr                         0
cap                                   0
GICS Sector ID                        0
GICS Sector                           0
GICS Industry Group ID                0


,symbol,cap,GICS Sector ID,GICS Sector,GICS Industry Group ID,GICS Industry Group,GICS Industry ID,GICS Industry,GICS Sub-Industry ID,GICS Sub-Industry,date,adjclose,TotalRevenue,OperatingRevenue,NetIncome,BasicEPS,DilutedEPS,EBITDA,EBIT,Current Ratio,Quick Ratio,Gross Profit Margin,Net Profit Margin,Cash Flow Margin,Operating Margin,ROA,CROA,ROE,Efficiency Ratio,Inventory Turnover,Debt to Equity,Debt Ratio,Interest Coverage,Cash Flow to Debt,Assets to Equity,R&D to Revenue,Investment CF to OCF,Financing CF to OCF,FreeCashFlow,Capex,Book Value Per Share,Tangible Book Value Per Share,fed_funds,treasury_10yr,treasury_2yr,baa_credit_spread,aaa_credit_spread,treasury_3mo,vix,gold,oil,silver,copper,aluminium,platinum,palladium,natural_gas,wheat,corn,soybean,coffee,cocao,sugar,cotton,mortgage_30yr,mortgage_15yr,cpi,unemployment,federal_budget_deficit,housing_starts,gdp_real,avg_home_price
6,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2014-01-31,0.016787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.121711,-0.105263,0.030043,0.150327,-0.841270,0.341837,0.031783,-0.009449,-0.012100,-0.064216,NaN,0.002188,-0.020073,0.168558,-0.081784,0.028436,-0.022667,0.130985,0.074566,-0.052407,0.014060,-0.035714,-0.034091,0.002972,-0.042857,NaN,-0.084326,0.020251,NaN
7,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2014-02-28,-0.020980,-0.413271,-0.413271,-0.075829,-0.078125,-0.079365,-0.414248,-0.556338,0.052895,0.05217,-0.042702,0.575124,-0.122953,-0.258452,-0.071659,-0.483089,-0.102637,0.038654,0.646016,-0.030446,0.003023,-0.541039,-0.484647,-0.033368,-0.13304,0.441616,-23.024055,-0.568116,0.40625,0.028574,0.18369,-0.142857,-0.003745,-0.029412,-0.020833,-0.056818,3.300000,-0.239544,0.065559,0.052313,0.109867,0.005589,NaN,0.052471,0.058108,-0.067570,0.077823,0.054147,0.102514,0.436102,0.008245,0.059164,0.008855,0.011574,-0.002941,0.001496,-0.014925,NaN,-0.119119,NaN,NaN
8,A,large,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352030.0,Life Sciences Tools & Services,35203010.0,Life Sciences Tools & Services,2014-03-31,-0.017741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.026316,0.333333,-0.038298,-0.042169,-0.348837,-0.008571,-0.028757,-0.009845,-0.069327,-0.059287,NaN,-0.019154,0.045238,-0.051638,0.164023,0.097268,0.035178,-0.010567,0.006814,0.078931,0.080032,0.006865,0.008850,0.001005,0.015152,NaN,0.030682,NaN,NaN
